In [266]:
import numpy as np
import pandas as pd
import random
# Parameters of the binary genetic algorithm
datafull=pd.read_csv('fullFrame_toy15.csv')
data=len(datafull.values)
iteration = 25
gen = 6 # number of bits for each variable
pop_size = 5
parent_size=2
crossover_rate = 0.8
mutation_rate = 0.01

RANDOM DECIMAL

In [267]:
k_decimal = []
tmp = np.random.randint(1, data)
for x in range(pop_size):
    while tmp in k_decimal:
        tmp = np.random.randint(1, data)  
    k_decimal.append(tmp)
k_decimal=np.sort(k_decimal)
print(k_decimal)

[2 3 4 8 9]


DECIMAL TO BINARY

In [268]:
k_binary=[] #k_binary -> kromosom
populasi=[]
for i in range(len(k_decimal)):
    k_binary.append(np.binary_repr(k_decimal[i], 6))
    gen=[]
    for j in range(len(k_binary[i])):
        gen.append(int(k_binary[i][j]))
    populasi.append(gen)
print(populasi)

[[0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 1, 1], [0, 0, 0, 1, 0, 0], [0, 0, 1, 0, 0, 0], [0, 0, 1, 0, 0, 1]]


READ EUCLIDEAN FROM CSV

In [269]:

datasetEuc=pd.read_csv('euclideanToy15_knn.csv')
euclidean=[]
euclidean_sorted=[]
indexSorted =[]
for x in range(len(datasetEuc.values)):
    to_be_removed_inds = [0]
    data = datasetEuc.values[x]
    data = data[[x for x in range(len(data)) if x not in to_be_removed_inds]]
    euclidean.append(data)
    data = np.argsort(data)
    indexSorted.append(data)
for x in range(len(indexSorted)):
    data=[]
    for y in range(len(indexSorted[x])):
        data.append(euclidean[x][y])
    euclidean_sorted.append(data)
# print(euclidean_sorted)


FITNESS

In [270]:
fitnes=[]
probFitnes=[]
for i in range(len(euclidean_sorted)):
    fitKol=[]
    pfKol=[]
    for j in range(len(k_decimal)): #meghitung nilai fitness
        # print(i,euclidean_sorted[i][0:k_decimal[j]],"->",1/sum(euclidean_sorted[i][0:k_decimal[j]]))
        fitKol.append(1/sum(euclidean_sorted[i][0:k_decimal[j]]))
    fitnes.append(fitKol)
    for j in range(len(fitnes[i])): #menghitung nilai probabilitas fitness
        # print(i,fitnes[i][j],sum(fitnes[i]),fitnes[i][j]/sum(fitnes[i]))
        pfKol.append(fitnes[i][j]/sum(fitnes[i]))
    probFitnes.append(pfKol)
# print("FITNESS\n",fitnes)
# print("\nPROBABILITAS FITNESS\n",probFitnes)

PENENTUAN DAERAH KROMOSOM

In [275]:
#probabilitask kumulatif
probKum=[]
for i in range(len(probFitnes)):
    pkKol=[]
    for j in range(len(probFitnes[i])):
        if(j==0): # print(i,j,probFitnes[i][j])
            pkKol.append(probFitnes[i][j])
        else: # print(i,j,sum(probFitnes[i][0:j])+probFitnes[i][j])
            pkKol.append(sum(probFitnes[i][0:j])+probFitnes[i][j])
    probKum.append(pkKol)

#daerah kromosom
daerahKromosom=[]
for i in range(len(probKum)):
    daerah=[]
    for j in range(len(probKum[i])):
        if(j==0):
            daerah.append([0,probKum[i][j]])
        else:
            awal=probKum[i][j-1]+0.0000000000001
            akhir=probKum[i][j]
            daerah.append([awal,akhir])
    daerahKromosom.append(daerah)
    # for j in range(len(daerahKromosom[i])):
    #     print("proses missing value ke",i,"-> Kromosom",j+1,daerahKromosom[i][j])

SELEKSI PARENTS

In [272]:
#RANDOM PARENT
random_value=[]
parent=[]
index_parent=[]
for i in range(len(daerahKromosom)):
    parentKol=[random.random(),random.random()]
    random_value.append(parentKol)
    # print("Proses missing value  ke",i,"-> nilai random parent",random_value[i])
    lpKol=[]
    for j in range(len(random_value[i])):
        for x in range(len(daerahKromosom[i])):
            if(random_value[i][j]>daerahKromosom[i][x][0] and random_value[i][j]<daerahKromosom[i][x][1]): # print("Field",i,"Parent",parent[i][j],"--> Kromosom",x+1)
                lpKol.append(x)#index daerah kromosom
            else:
                continue
    index_parent.append(i)
    parent.append(lpKol)#index kromosom / daerah kromosom
print(parent)
print(index_parent) #index urut berdasarkan tiap proses missing value

[[4, 3], [2, 0], [2, 0], [1, 4], [2, 1]]
[0, 1, 2, 3, 4]


CROSSOVER

In [273]:
range_to_crossover= int(crossover_rate*len(index_parent))
rand_idx = random.sample(index_parent,range_to_crossover) #yang dilakukan random adalah indexnya
print(rand_idx)
for i in range(len(rand_idx)):
    index_crossover=parent[rand_idx[i]]
    print(i,parent[rand_idx[i]],">>",index_crossover[0],populasi[index_crossover[0]],"|",index_crossover[1],populasi[index_crossover[1]])
    to_be_crossover1,to_be_crossover2=[],[]
    to_be_crossover1+=populasi[index_crossover[0]]
    to_be_crossover2+=populasi[index_crossover[1]]

    child1,child2=[],[]
    child1+=populasi[index_crossover[0]]
    child2+=populasi[index_crossover[1]]

    print("to be co 1",to_be_crossover1,">>","to be co2",to_be_crossover2)
    # c1
    child1[:2]=to_be_crossover2[:2]
    child1[5:]=to_be_crossover2[5:]
    # c2
    child2[:2]=to_be_crossover1[:2]
    child2[5:]=to_be_crossover1[5:]
    print(">> result",child1,">>",child2,"\n")

[0, 1, 4, 3]
0 [4, 3] >> 4 [0, 0, 1, 0, 0, 1] | 3 [0, 0, 1, 0, 0, 0]
to be co 1 [0, 0, 1, 0, 0, 1] >> to be co2 [0, 0, 1, 0, 0, 0]
>> result [0, 0, 1, 0, 0, 0] >> [0, 0, 1, 0, 0, 1] 

1 [2, 0] >> 2 [0, 0, 0, 1, 0, 0] | 0 [0, 0, 0, 0, 1, 0]
to be co 1 [0, 0, 0, 1, 0, 0] >> to be co2 [0, 0, 0, 0, 1, 0]
>> result [0, 0, 0, 1, 0, 0] >> [0, 0, 0, 0, 1, 0] 

2 [2, 1] >> 2 [0, 0, 0, 1, 0, 0] | 1 [0, 0, 0, 0, 1, 1]
to be co 1 [0, 0, 0, 1, 0, 0] >> to be co2 [0, 0, 0, 0, 1, 1]
>> result [0, 0, 0, 1, 0, 1] >> [0, 0, 0, 0, 1, 0] 

3 [1, 4] >> 1 [0, 0, 0, 0, 1, 1] | 4 [0, 0, 1, 0, 0, 1]
to be co 1 [0, 0, 0, 0, 1, 1] >> to be co2 [0, 0, 1, 0, 0, 1]
>> result [0, 0, 0, 0, 1, 1] >> [0, 0, 1, 0, 0, 1] 

